In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
combined_data_df = pd.read_csv('combined_data_1.csv')
combined_data_df.head()

,side_a_2nd,side_b_2nd,incompatibility,cumulative_intensity,type_of_conflict,ep_end,region,S&P_500,NASDAQ,Dow_Jones,...,territory_name_Other,territory_name_Punjab/Khalistan,territory_name_Rojava Kurdistan,territory_name_Serb,territory_name_Southern Lebanon,territory_name_Tripura,Date,year,month,day
0,0,0,1,0,3,1,3,1359.88,2853.13,12588.31,...,1,0,0,0,0,0,11/15/2012,2012,11,15
1,0,0,1,0,3,1,3,1985.44,4485.93,17068.26,...,1,0,0,0,0,0,7/1/2014,2014,7,1
2,0,0,1,0,3,1,4,1333.27,2803.32,12512.04,...,1,0,0,0,0,0,5/19/2011,2011,5,19
3,0,0,2,0,3,0,4,1123.53,2341.84,10817.65,...,1,0,0,0,0,0,8/20/2011,2011,8,20
4,0,0,2,0,3,0,4,1123.53,2341.84,10817.65,...,1,0,0,0,0,0,8/20/2011,2011,8,20


In [2]:
# Generate our categorical variable list
combined_cat = combined_data_df.dtypes[combined_data_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
combined_data_df[combined_cat].nunique()

side_a_2nd      48
side_b_2nd      12
region          10
Date          2107
dtype: int64

In [3]:
# Check the unique value counts to see if binning is required
combined_data_df.Date.value_counts()

2/12/1990     31
8/15/1984     30
12/9/1991     28
12/31/1994    22
12/31/1977    20
              ..
7/1/1988       1
6/24/1988      1
6/17/1988      1
6/10/1988      1
8/25/2017      1
Name: Date, Length: 2107, dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(combined_data_df[combined_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(combined_cat)
encode_df.head()

C:\Users\jeush\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,side_a_2nd_0,side_a_2nd_Government of Afghanistan,side_a_2nd_Government of Angola,"side_a_2nd_Government of Angola, Government of Chad","side_a_2nd_Government of Armenia, Government of Australia, Government of Austria, Government of Bangladesh, Government of Belgium, Government of Benin, Government of Bhutan, Government of Bosnia-Herzegovina, Government of Burkina Faso, Government of Burundi, Government of Cambodia (Kampuchea), Government of Cameroon, Government of Canada, Government of Chad, Government of China, Government of Czech Republic, Government of Denmark, Government of Egypt, Government of El Salvador, Government of Estonia, Government of Ethiopia, Government of Finland, Government of France, Government of Gambia, Government of Germany, Government of Ghana, Government of Guatemala, Government of Guinea, Government of Indonesia, Government of Iran, Government of Ireland, Government of Italy, Government of Ivory Coast, Government of Jordan, Government of Kenya, Government of Latvia, Government of Liberia, Government of Lithuania, Government of Madagascar, Government of Mauritania, Government of Mexico, Government of Nepal, Government of Netherlands, Government of Niger, Government of Nigeria, Government of Norway, Government of Pakistan, Government of Portugal, Government of Romania, Government of Senegal, Government of Sierra Leone, Government of Spain, Government of Sri Lanka, Government of Sweden, Government of Switzerland, Government of Togo, Government of Tunisia, Government of Ukraine, Government of United Kingdom, Government of United States of America, Government of Zambia","side_a_2nd_Government of Armenia, Government of Austria, Government of Bangladesh, Government of Belgium, Government of Benin, Government of Bhutan, Government of Bosnia-Herzegovina, Government of Burkina Faso, Government of Burundi, Government of Cambodia (Kampuchea), Government of Cameroon, Government of Canada, Government of Chad, Government of China, Government of Czech Republic, Government of Denmark, Government of Egypt, Government of El Salvador, Government of Estonia, Government of Ethiopia, Government of Finland, Government of France, Government of Gambia, Government of Germany, Government of Ghana, Government of Guatemala, Government of Guinea, Government of Hungary, Government of Indonesia, Government of Italy, Government of Ivory Coast, Government of Jordan, Government of Kenya, Government of Latvia, Government of Liberia, Government of Lithuania, Government of Madagascar, Government of Mauritania, Government of Mexico, Government of Nepal, Government of Netherlands, Government of New Zealand, Government of Niger, Government of Nigeria, Government of Norway, Government of Pakistan, Government of Portugal, Government of Romania, Government of Senegal, Government of Sierra Leone, Government of Spain, Government of Sri Lanka, Government of Sweden, Government of Switzerland, Government of Togo, Government of Tunisia, Government of Turkey, Government of United Kingdom, Government of United States of America","side_a_2nd_Government of Armenia, Government of Austria, Government of Bangladesh, Government of Belgium, Government of Benin, Government of Bhutan, Government of Bosnia-Herzegovina, Government of Burkina Faso, Government of Burundi, Government of Cambodia (Kampuchea), Government of Cameroon, Government of Canada, Government of Chad, Government of China, Government of Czech Republic, Government of Denmark, Government of Egypt, Government of El Salvador, Government of Estonia, Government of Ethiopia, Government of Finland, Government of France, Government of Gambia, Government of Germany, Government of Ghana, Government of Guatemala, Government of Guinea, Government of Indonesia, Government of Ireland, Government of Italy, Government of Ivory Coast, Government of Jordan, Government of Kenya, Government of Latvia, Government of Liberia, Government of Lithuania, Government of Madagascar, Government of Mauritania, Government of Mex

In [5]:
# Merge one-hot encoded features and drop the originals
combined_data_df = combined_data_df.merge(encode_df,left_index=True, right_index=True)
combined_data_df = combined_data_df.drop(combined_cat,1)
combined_data_df.head()

C:\Users\jeush\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,incompatibility,cumulative_intensity,type_of_conflict,ep_end,S&P_500,NASDAQ,Dow_Jones,region_1_x,region_2_x,region_3_x,...,Date_9/8/2006,Date_9/8/2018,Date_9/9/1977,Date_9/9/1983,Date_9/9/1988,Date_9/9/1994,Date_9/9/2005,Date_9/9/2010,Date_9/9/2011,Date_9/9/2016
0,1,0,3,1,1359.88,2853.13,12588.31,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,3,1,1985.44,4485.93,17068.26,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,3,1,1333.27,2803.32,12512.04,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,0,3,0,1123.53,2341.84,10817.65,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,0,3,0,1123.53,2341.84,10817.65,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Remove incompatibility target from features data
y = combined_data_df.incompatibility
X = combined_data_df.drop(columns=["incompatibility"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.986


In [13]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
61/61 [==============================] - 1s 2ms/step - loss: 0.4608 - accuracy: 0.6941
Epoch 2/50
61/61 [==============================] - 0s 2ms/step - loss: -0.5093 - accuracy: 0.8727
Epoch 3/50
61/61 [==============================] - 0s 3ms/step - loss: -2.7282 - accuracy: 0.8659
Epoch 4/50
61/61 [==============================] - 0s 3ms/step - loss: -8.5488 - accuracy: 0.8540
Epoch 5/50
61/61 [==============================] - 0s 2ms/step - loss: -20.9778 - accuracy: 0.8463
Epoch 6/50
61/61 [==============================] - 0s 2ms/step - loss: -42.7341 - accuracy: 0.8520
Epoch 7/50
61/61 [==============================] - 0s 2ms/step - loss: -77.5506 - accuracy: 0.8478
Epoch 8/50
61/61 [==============================] - 0s 2ms/step - loss: -127.2288 - accuracy: 0.8540
Epoch 9/50
61/61 [==============================] - 0s 2ms/step - loss: -193.0518 - accuracy: 0.8411
Epoch 10/50
61/61 [==============================] - 0s 2ms/step - loss: -279.5719 - accuracy: 0.8504
E